* A simple single head attention class that implents causal self attention with dropout
* And then a naive implementation of a multihead attention class by "stacking" together single head attention modules

In [8]:
import torch
import torch.nn as nn

class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length),
            diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1,2)
        attn_scores.masked_fill(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vectors = attn_weights@values
        return context_vectors


In [11]:
#create a batch of inputs by stacking our inputs over itself
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

d_in = inputs.shape[1] ## the input embedding size
d_out = 2 ## the output embedding size

batch = torch.stack((inputs,inputs),dim=0)

causal_attn = CausalAttention(d_in, d_out, batch.shape[1], 0)
context_vecs = causal_attn(batch)
context_vecs.shape, context_vecs

(torch.Size([2, 6, 2]),
 tensor([[[ 0.0441, -0.3124],
          [ 0.0449, -0.3138],
          [ 0.0450, -0.3137],
          [ 0.0506, -0.3147],
          [ 0.0504, -0.3132],
          [ 0.0489, -0.3152]],
 
         [[ 0.0441, -0.3124],
          [ 0.0449, -0.3138],
          [ 0.0450, -0.3137],
          [ 0.0506, -0.3147],
          [ 0.0504, -0.3132],
          [ 0.0489, -0.3152]]], grad_fn=<UnsafeViewBackward0>))

A multihead attention class with dropout that wraps the Causal Attention class from above

In [14]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, 
                 dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
                        [CausalAttention(d_in, d_out, context_length,
                                            dropout, qkv_bias)
                        for _ in range(num_heads)])

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


In [17]:
torch.manual_seed(123)
multi_head_attn = MultiHeadAttention(d_in, d_out, batch.shape[1], 0, 2)

context_vecs = multi_head_attn(batch)
context_vecs.shape, context_vecs

(torch.Size([2, 6, 4]),
 tensor([[[-0.5337, -0.1051,  0.5085,  0.3508],
          [-0.5323, -0.1080,  0.5084,  0.3508],
          [-0.5323, -0.1079,  0.5084,  0.3506],
          [-0.5297, -0.1076,  0.5074,  0.3471],
          [-0.5311, -0.1066,  0.5076,  0.3446],
          [-0.5299, -0.1081,  0.5077,  0.3493]],
 
         [[-0.5337, -0.1051,  0.5085,  0.3508],
          [-0.5323, -0.1080,  0.5084,  0.3508],
          [-0.5323, -0.1079,  0.5084,  0.3506],
          [-0.5297, -0.1076,  0.5074,  0.3471],
          [-0.5311, -0.1066,  0.5076,  0.3446],
          [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>))